In [1]:
%load_ext autoreload
%autoreload
from utils.utils import make_data_set, read_pkl, save_pkl, make_recipes,filter_recipe
from utils.build_chroma_db import build_chroma_db, requst_chroma_db

from utils.Recipe import Recipe, RecipesProject
import torch

/workspace/llm_venv/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/workspace/llm_venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
EMBEDDING_MODEL = "ai-forever/sbert_large_nlu_ru"
def create_db(recipes_list):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    vector_store, tokenizer = build_chroma_db(
        recipes_list,
        embedding_model=EMBEDDING_MODEL, #"nomic-ai/nomic-embed-text-v1.5", #"IlyaGusev/saiga_yandexgpt_8b" 
        device=device,
    )
    return vector_store, tokenizer

recipes_list = read_pkl("./data/rp_recipes.pickle")      

vector_store, tokenizer = create_db(recipes_list)
  
knowledgeGraph = read_pkl("./data/rp_knowledgeGraph.pickle")        
tags = read_pkl("./data/rp_tags.pickle")
oneWordTags = read_pkl("./data/rp_oneWordTags.pickle")  

rp = RecipesProject(
    recipes=recipes_list,
    knowledgeGraph=knowledgeGraph,
    tags=tags,
    vectorStore=vector_store,
    oneWordTags=oneWordTags
)

Collected 32330 recipes
39813 chunks are to be loaded to chroma db
Collected 33968 recipes
Collected 1757 recipes
Collected 708 recipes


In [3]:
%load_ext autoreload
%autoreload
from utils.build_ollama import build_model, llm_invoke
llm = build_model()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
requests = [
    ["Какой рецепт вегетарианской пасты?"],
    ["Как приготовить блинчики?"],
    ["Что можно приготовить из куриного филе?"],
    ["Как сделать шоколадный торт?"],
    ["Как приготовить рис для суши?"],
    ["Как приготовить суп-пюре из тыквы?"],
    ["Что можно приготовить из картошки и сыра?"],
    ["Как сделать домашний хлеб?"],
    ["Какие десерты можно приготовить без муки?"],
    ["Как приготовить рис с курицей и овощами?"],
    ["Как сделать соус для пасты?"],
    ["Как приготовить морковные котлеты?"],
    ["Что приготовить на ужин за 30 минут?"],
    ["Как сделать домашнюю пиццу?"],
    ["Что можно приготовить из куриного филе, риса и брокколи?"],
    ["Как приготовить сладкий омлет?"],
    ["Как сделать домашнее мороженое без мороженицы?"],
    ["Что можно приготовить для обеда из мяса и риса?"],
    ["Как приготовить крем-суп из грибов?"],
    ["Как приготовить рыбу с картошкой в духовке?"],
    ["Что приготовить для завтрака, если есть яйца, авокадо и помидоры?"],
    ["Как приготовить куриные крылышки на гриле?"],
    ["Что приготовить на праздничный ужин?"],
    ["Как приготовить соус бешамель?"],
    ["Как приготовить кальмары с овощами ?"],
]

In [5]:
recipes_dict = {r.id: r for r in rp.recipes}

In [6]:
import random
device = "cuda" if torch.cuda.is_available() else "cpu"
def final_query(query, n=3):
    query_clean = query.lower().replace("рецепт","").replace("кухня","")
    results_kg = rp.invoke(query_clean, verbose=False)
    if len(results_kg[0]) == 0:
        results_kg = results_kg[1]
    else:
        results_kg = results_kg[0]
    id_kg = [id for id in results_kg]

    results_db = requst_chroma_db(vector_store, tokenizer, query, device, k=100)
    id_db = [r.metadata['id'] for r in results_db]
    
    res_final = []
    for id in results_db:
        if id in id_kg:
            res_final.append(id)
    if len(res_final) < n:
        kg_rest = [id for id in results_kg if id not in id_db]
        res_final.extend(random.sample(kg_rest, k=min([n-len(res_final), len(kg_rest)])))
    if len(res_final) < n:
        db_rest = [id for id in id_db if id not in res_final]
        res_final.extend(db_rest[0:n-len(res_final)])
        
    return id_kg[:n], id_db[:n], res_final[:n]

In [7]:
%load_ext autoreload
%autoreload
from prompt import create_chat_history, prompt, formatted_chat_history, formatted_rag
chat_history = create_chat_history()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import time
from tqdm import tqdm

log = []

for query in tqdm(requests[:10]):
    log.append([])
    chat_history = create_chat_history()
    for question in query:
        start_time = time.time()
        kg, db, final = final_query(question)
        format_chat_history = formatted_chat_history(chat_history)
        format_rag = formatted_rag([recipes_dict[r].display for r in final])
        question_input = {
            "chat_history": format_chat_history,
            "user_input": question,
            "retrieved_recipes": format_rag
        }
        final_prompt = prompt.format(**question_input)
        llm_prompt_result = llm_invoke(llm, final_prompt)
        llm_prompt_time = time.time() - start_time
        start_time = time.time()
        llm_question_result = llm_invoke(llm, question)
        llm_question_time = time.time() - start_time
        chat_history.add_user_message(question)
        chat_history.add_ai_message(llm_prompt_result)
        log[-1].append({
            'kg':kg,
            'db':db,
            'llm_question':llm_question_result,
            'time_question':llm_question_time,
            'llm_prompt':llm_prompt_result,
            'time_prompt':llm_prompt_time,
        })

100%|██████████| 10/10 [02:01<00:00, 12.10s/it]


In [49]:
ground_trouth = ["""
### 🍝 **Вегетарианская паста с овощами и томатным соусом**  

#### **Ингредиенты**  
- Паста (спагетти или пенне) — 250 г  
- Оливковое масло — 2 ст. ложки  
- Помидоры (консервированные или свежие) — 400 г  
- Чеснок — 2 зубчика (измельчить)  
- Лук — 1 шт. (мелко нарезать)  
- Кабачок — 1 шт. (нарезать кубиками)  
- Перец сладкий — 1 шт. (нарезать полосками)  
- Базилик (свежий или сушёный) — 1 ч. ложка  
- Орегано — 1 ч. ложка  
- Соль и перец — по вкусу  

#### **Инструкция**  
1. **Варка пасты**: Отварите пасту в подсоленной воде до состояния *аль денте*.  
2. **Приготовление соуса**:  
   - В сковороде разогрейте оливковое масло.  
   - Добавьте лук и чеснок, обжаривайте до прозрачности (около 2 минут).  
   - Добавьте перец и кабачок, обжаривайте 5–7 минут.  
   - Влейте помидоры, добавьте базилик, орегано, соль и перец.  
   - Томите на слабом огне 10 минут, периодически помешивая.  
3. **Смешивание**: Добавьте пасту в соус, тщательно перемешайте.  
4. **Подача**: Подавайте горячей, посыпав свежим базиликом или тёртым пармезаном (по желанию).  

#### **Советы/Альтернативы**  
✅ Можно добавить оливки или каперсы для более насыщенного вкуса.  
✅ Кабачок можно заменить на баклажан или брокколи.  
✅ Вместо пасты попробуйте использовать цельнозерновые макароны или безглютеновые варианты.  

""",
"""
### 🥞 **Классические блинчики**  

#### **Ингредиенты**  
- Мука — 150 г  
- Молоко — 300 мл  
- Яйцо — 2 шт.  
- Сахар — 1 ст. ложка  
- Соль — щепотка  
- Растительное масло — 2 ст. ложки  

#### **Инструкция**  
1. **Приготовление теста**:  
   - В миске взбейте яйца с сахаром и солью.  
   - Добавьте молоко, затем постепенно добавляйте муку, постоянно взбивая, чтобы не было комков.  
   - Влейте масло, перемешайте до однородности.  
2. **Жарка**:  
   - Разогрейте сковороду, смажьте небольшим количеством масла.  
   - Вылейте немного теста, равномерно распределяя его по сковороде.  
   - Жарьте 1–2 минуты с каждой стороны до золотистой корочки.  
3. **Подача**: Подавайте с мёдом, джемом или сметаной.  

#### **Советы/Альтернативы**  
✅ Можно добавить ванильный сахар для аромата.  
✅ Если тесто густое — добавьте немного молока.  
✅ Для безглютенового варианта используйте рисовую или овсяную муку.  

""",
"""
### 🍗 **Куриное филе в сливочном соусе**  

#### **Ингредиенты**  
- Куриное филе — 2 шт.  
- Сливки (20–30%) — 200 мл  
- Чеснок — 2 зубчика  
- Сливочное масло — 30 г  
- Соль и перец — по вкусу  
- Петрушка — для украшения  

#### **Инструкция**  
1. Нарежьте куриное филе на небольшие куски.  
2. Разогрейте сливочное масло в сковороде, обжарьте филе до румяной корочки (5–7 минут).  
3. Добавьте измельчённый чеснок, жарьте ещё 1 минуту.  
4. Влейте сливки, посолите и поперчите. Томите на слабом огне 10 минут.  
5. Подавайте, посыпав петрушкой.  

#### **Советы/Альтернативы**  
✅ Можно добавить грибы или шпинат в соус.  
✅ В сливки можно добавить немного тёртого пармезана для насыщенности.  

""",
"""
### 🍫 **Шоколадный торт без выпечки**  

#### **Ингредиенты**  
- Печенье — 200 г  
- Сливочное масло — 100 г  
- Тёмный шоколад — 200 г  
- Сливки (30%) — 150 мл  

#### **Инструкция**  
1. Измельчите печенье в крошку, смешайте с растопленным маслом.  
2. Утрамбуйте в форму, охладите 30 минут.  
3. Растопите шоколад на водяной бане, влейте сливки, перемешайте.  
4. Вылейте на основу из печенья.  
5. Охладите в холодильнике 2 часа.  

#### **Советы/Альтернативы**  
✅ Можно добавить орехи или кокосовую стружку в основу.  
✅ Вместо тёмного шоколада используйте молочный или белый.  

""",
"""
### 🍣 **Рис для суши**  

#### **Ингредиенты**  
- Рис для суши — 250 г  
- Вода — 300 мл  
- Рисовый уксус — 3 ст. ложки  
- Сахар — 1 ст. ложка  
- Соль — 0,5 ч. ложки  

#### **Инструкция**  
1. **Подготовка риса**  
   - Промойте рис в холодной воде 3–4 раза, пока вода не станет прозрачной.  
   - Оставьте рис в воде на 30 минут, затем слейте воду.  

2. **Варка риса**  
   - Выложите рис в кастрюлю, залейте водой (соотношение риса к воде 1:1,2).  
   - Доведите до кипения, затем убавьте огонь и готовьте под крышкой 10–12 минут.  
   - Снимите с огня и оставьте под крышкой ещё 10 минут для пропаривания.  

3. **Заправка**  
   - В небольшой миске смешайте рисовый уксус, сахар и соль.  
   - Осторожно влейте смесь в рис и аккуратно перемешайте деревянной лопаткой, чтобы не повредить зёрна.  

4. **Охлаждение**  
   - Оставьте рис остывать при комнатной температуре, накрыв влажным полотенцем, чтобы рис не пересох.  

#### **Советы/Альтернативы**  
✅ Используйте деревянную или стеклянную посуду для перемешивания риса (не металлическую).  
✅ Если нет рисового уксуса, смешайте яблочный уксус с небольшим количеством сахара и соли.  
✅ Не промешивайте рис слишком сильно — зёрна должны оставаться целыми.  
""",
"""### 🎃 **Суп-пюре из тыквы**  

#### **Ингредиенты**  
- Тыква — 500 г (очищенная и нарезанная кубиками)  
- Картофель — 2 шт. (нарезать кубиками)  
- Морковь — 1 шт. (нарезать кружками)  
- Лук — 1 шт. (мелко нарезать)  
- Овощной бульон — 500 мл  
- Сливки (10–20%) — 150 мл  
- Сливочное масло — 20 г  
- Соль и перец — по вкусу  
- Мускатный орех — щепотка (по желанию)  

#### **Инструкция**  
1. **Обжарка овощей**  
   - В кастрюле растопите сливочное масло.  
   - Добавьте лук и обжаривайте до прозрачности (около 3 минут).  
   - Добавьте морковь и картофель, жарьте 5 минут.  

2. **Варка**  
   - Добавьте тыкву и залейте овощным бульоном.  
   - Доведите до кипения, уменьшите огонь и готовьте под крышкой 20 минут до мягкости овощей.  

3. **Пюрирование**  
   - С помощью блендера измельчите суп до однородности.  
   - Добавьте сливки, соль, перец и мускатный орех.  
   - Прогрейте суп на слабом огне (не доводя до кипения).  

4. **Подача**  
   - Подавайте горячим, украсив зеленью или тыквенными семечками.  

#### **Советы/Альтернативы**  
✅ Для более насыщенного вкуса добавьте немного имбиря или чеснока при обжарке.  
✅ Если суп получился слишком густым, разбавьте его бульоном или водой.  
✅ Вместо сливок можно использовать кокосовое молоко для веганской версии.  

""",
"""
### 🍠 **Картофельные вафли с сыром**  

#### **Ингредиенты**  
- Картофель — 2 шт.  
- Яйцо — 1 шт.  
- Сыр (тёртый) — 100 г  
- Мука — 2 ст. ложки  
- Соль и перец — по вкусу  

#### **Инструкция**  
1. Натрите картофель на мелкой тёрке, отожмите лишний сок.  
2. Смешайте с яйцом, сыром, мукой, солью и перцем.  
3. Выпекайте в вафельнице до золотистой корочки (5–7 минут).  

#### **Советы/Альтернативы**  
✅ Можно добавить зелень (укроп, петрушку).  
✅ Если нет вафельницы — используйте сковороду.  

""",
"""
### 🍞 **Домашний хлеб**  

#### **Ингредиенты**  
- Мука — 500 г  
- Вода — 300 мл  
- Дрожжи — 7 г (сухие)  
- Соль — 1 ч. ложка  
- Сахар — 1 ч. ложка  

#### **Инструкция**  
1. Смешайте дрожжи с тёплой водой и сахаром, оставьте на 10 минут.  
2. Добавьте муку, соль и замесите тесто.  
3. Оставьте в тёплом месте на 1 час.  
4. Выложите тесто в форму, дайте подняться ещё 30 минут.  
5. Выпекайте при 180°C 30 минут.  

#### **Советы/Альтернативы**  
✅ Можно добавить орехи, семечки или оливки в тесто.  
✅ Для хрустящей корочки смажьте верх водой перед выпечкой.  
""",
"""
### 🍫 **Десерты без муки**  

Вот несколько простых и вкусных десертов без муки:  

---
### 🍮 **Шоколадный мусс**  

#### **Ингредиенты**  
- Тёмный шоколад (70%) — 200 г  
- Яйца — 3 шт. (разделить на белки и желтки)  
- Сахар — 50 г  
- Взбитые сливки (30%) — 200 мл  
- Ванильный экстракт — 1 ч. ложка  

#### **Инструкция**  
1. **Растопите шоколад** на водяной бане или в микроволновке (на низкой мощности), дайте немного остыть.  
2. Взбейте желтки с сахаром до светлой массы, добавьте растопленный шоколад и ванильный экстракт.  
3. В отдельной миске взбейте белки до устойчивых пиков.  
4. Аккуратно введите белки и взбитые сливки в шоколадную смесь.  
5. Разлейте по формам и охладите в холодильнике минимум на 2 часа.  

#### **Советы/Альтернативы**  
✅ Добавьте щепотку соли в белки для лучшего взбивания.  
✅ Вместо сливок можно использовать кокосовые сливки для веганской версии.  
✅ Украсьте ягодами или шоколадной крошкой.  
""",
"""
### 🍚 **Рис с курицей и овощами**  

#### **Ингредиенты**  
- Рис — 200 г  
- Куриное филе — 150 г  
- Морковь — 1 шт.  
- Горошек (замороженный) — 100 г  
- Соевый соус — 2 ст. ложки  

#### **Инструкция**  
1. Отварите рис до готовности.  
2. Обжарьте курицу на сковороде (5–7 минут).  
3. Добавьте морковь и горошек, жарьте 5 минут.  
4. Влейте соевый соус, добавьте рис, перемешайте.  
5. Жарьте ещё 2–3 минуты.  

#### **Советы/Альтернативы**  
✅ Добавьте яйца для версии «жареного риса».  
✅ Можно использовать креветки вместо курицы.  
""",
]

In [125]:
ground_trouth_cleaned = []
for ans in ground_trouth:
    ground_trouth_cleaned.append(re.sub( "[^А-Яа-я0-9 ,-.]+", "", ans).strip())
    

In [126]:
import pandas as pd
df = pd.DataFrame()
for l in log:
    df = pd.concat([df, pd.DataFrame.from_dict(l)], axis=0)

In [127]:
df['kg_resp'] = df.apply(lambda x: recipes_dict[x['kg'][0]].display, axis=1)
df['db_resp'] = df.apply(lambda x: recipes_dict[x['db'][0]].display, axis=1)

In [128]:
df['request'] = requests[:10]
df['ground_trouth'] = ground_trouth_cleaned

In [129]:
df.reset_index(inplace=True)

### Scoring

In [130]:
from evaluate import load

rouge = load('rouge')
bertscore = load('bertscore')

for resp in ['llm_question', 'llm_prompt', 'kg_resp', 'db_resp']:
    references = list(df['ground_trouth'])
    predictions = list(df[resp])
    results = bertscore.compute(predictions=predictions, references=references, lang="ru")

    results = pd.DataFrame(results)
    results.columns = [col+"_"+resp for col in results.columns]
    df = pd.concat([df, results], axis=1)

precision_llm_question    0.716637
recall_llm_question       0.701438
f1_llm_question           0.708675
precision_llm_prompt      0.648860
recall_llm_prompt         0.668850
f1_llm_prompt             0.658613
precision_kg_resp         0.677914
recall_kg_resp            0.698647
f1_kg_resp                0.687902
precision_db_resp         0.659009
recall_db_resp            0.683721
f1_db_resp                0.671043
dtype: float64

In [131]:
df[[ 'precision_llm_question', 'recall_llm_question', 'f1_llm_question',
       'precision_llm_prompt', 'recall_llm_prompt','f1_llm_prompt', 
     'precision_kg_resp','recall_kg_resp', 'f1_kg_resp', 
     'precision_db_resp','recall_db_resp', 'f1_db_resp']].mean()

precision_llm_question    0.716637
recall_llm_question       0.701438
f1_llm_question           0.708675
precision_llm_prompt      0.648860
recall_llm_prompt         0.668850
f1_llm_prompt             0.658613
precision_kg_resp         0.677914
recall_kg_resp            0.698647
f1_kg_resp                0.687902
precision_db_resp         0.659009
recall_db_resp            0.683721
f1_db_resp                0.671043
dtype: float64

In [132]:
df.to_csv("./data/result.csv")